In [2]:
from pandas import ExcelWriter
from pandas import ExcelFile
from datetime import date
import json
import pandas as pd
import os
import datetime

now = datetime.datetime.now()

#Verifica diretório Atual 
os.getcwd() 

#Configura Diretório
os.chdir('C:\\Users\\filip\\OneDrive\\Documentos\\Python\\Arquivos') 

In [33]:
#Trata arquivo Bruto
dfFinal = pd.read_csv("consCodUOL.csv", sep = ";")

dfFinal.drop(dfFinal.columns[[0,3]], axis=1, inplace = True)

dfConsolidadp = pd.read_csv("CONSOLIDADO.csv", sep = ";")

dfConsolidadp["Volume"] = (dfConsolidadp["Volume"].str.replace(" ", "").str.split(".", expand = True)[0]
+ dfConsolidadp["Volume"].str.replace(" ", "").str.split(".", expand = True)[1].str[0:3].str.ljust(3,'0').fillna('')
+ dfConsolidadp["Volume"].str.replace(" ", "").str.split(".", expand = True)[2].str[0:3].str.ljust(3,'0').fillna('')
#+ dfConsolidadp["Volume"].str.replace(" ", "").str.split(".", expand = True)[3].str[0:3].fillna('')
)

dfFinal = dfFinal.merge(dfConsolidadp.loc[:,["cvm", "ano", "QtdAcoes", "Volume"]], on = ["cvm", "ano"], how = "inner")

for index, row in dfFinal.iterrows():
    if row["QtdAcoes"][-1] == " ":
        dfFinal.loc[index, "QtdAcoes"] = row["QtdAcoes"].replace(" ", "").replace(".", "")
    else:
        p = row["QtdAcoes"].split(".")
        p[-1] = p[-1][0:3].ljust(3,'0')
        if len(p) > 3:
            dfFinal.loc[index, "QtdAcoes"] =(int(p[0] + p[1] + p[2] + p[3]) * 1000)
        elif len(p) > 2:
            dfFinal.loc[index, "QtdAcoes"] =(int(p[0] + p[1] + p[2]) * 1000)
        elif len(p) > 1:
            dfFinal.loc[index, "QtdAcoes"] =(int(p[0] + p[1]) * 1000)
        else:
            dfFinal.loc[index, "QtdAcoes"] =(int(p[0]) * 1000)

#Retira Pontos
for c in ['1', '1.01.01', '1.01.02', '1.02.01.01',
       '1.02.01.02', '1.02.01.03', '2.01.04', '2.01.02', '2.02.01', '3.05']:
    dfFinal[c] = pd.to_numeric(dfFinal[c].str.replace(".", ""))

dfFinal["QtdAcoes"] = pd.to_numeric(dfFinal["QtdAcoes"])
dfFinal["open"] = pd.to_numeric(dfFinal["open"].str.replace(".", "").str.replace(",", "."))
dfFinal["close"] = pd.to_numeric(dfFinal["close"].str.replace(".", "").str.replace(",", "."))

dfFinal.fillna(value = 0, axis = 1, inplace = True)

dfFinal["EBIT"] = dfFinal["3.05"]
dfFinal["ROIC"] = dfFinal["EBIT"] / (dfFinal["1"] - dfFinal["2.01.02"] - dfFinal["1.01.01"])
dfFinal["EV"] = dfFinal["QtdAcoes"] * dfFinal["close"]
dfFinal["DVIDALIQUIDA"] = dfFinal["2.01.04"] + dfFinal["2.02.01"] - dfFinal["1.01.01"] - dfFinal["1.01.02"] - dfFinal["1.02.01.01"] - dfFinal["1.02.01.02"] - dfFinal["1.02.01.03"]

#Exporta o Excel com a formula mágica
dfFinal.to_excel("dados_bolsa_full_"+now.strftime("%Y-%m-%d")+".xlsx")

In [18]:
#Importa tabela de Ticker que deve estar no diretório de Trabalho
dados_bolsa = pd.read_excel('BolsaConsolidadoSaneado.xlsx', sheet_name='Planilha1')
dados_bolsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 10 columns):
codigo          1947 non-null object
ano             1947 non-null int64
open            1947 non-null float64
close           1947 non-null float64
QtdAcoes        1947 non-null int64
Volume          1947 non-null int64
EBIT            1947 non-null int64
ROIC            1947 non-null float64
EV              1947 non-null float64
DVIDALIQUIDA    1947 non-null int64
dtypes: float64(4), int64(5), object(1)
memory usage: 152.2+ KB


In [19]:
###Calcula Indicadores
dados_bolsa.DVIDALIQUIDA = dados_bolsa.DVIDALIQUIDA.fillna(0)

dados_bolsa['EV'] = dados_bolsa['QtdAcoes']*dados_bolsa['close']

#EV + Divida Liquida
dados_bolsa['EV_DIVIDAS'] = dados_bolsa['EV']+dados_bolsa['DVIDALIQUIDA']

#EV/EBIT
dados_bolsa['EV/EBIT'] = dados_bolsa['EV_DIVIDAS']/dados_bolsa['EBIT']

index_table = dados_bolsa[['codigo','ano','Volume','close','QtdAcoes','ROIC','EV','EV/EBIT']]

index_table.columns = ['TICKER','ANO','VOL_MEDIO','COTACAO','NUM_ACOES','ROIC','EV','EV/EBIT']

index_table = index_table.sort_values(by=['ANO','TICKER'],ascending=True) 

In [24]:
########## Início do Programa ########## 
#Variáveis utilizadas
Cont1 = 0
Volume = 50000
QtdAcoes = 30
AnoInicio = 2009
AnoAtual = 2018
Ano = AnoInicio-1
RetiraRepCot = True
GeraPrecoRelatorio = True
GeraCotacaoRelatorio = True
year_stocks = []
my_stocks = []
range_stocks = []
log_table = pd.DataFrame(columns=['SEQ','LOG'])

#Loga Programa
Cont1 += 1
log_table = log_table.append({'SEQ': Cont1,'LOG':'Início: '+str(datetime.datetime.now())}, ignore_index=True)

while Ano != AnoAtual:
    Ano += 1
    
    print(" ")
    print(">>>>>>>>>>>>>>>Ano: "+str(Ano))
    
    #Loga Programa
    Cont1 += 1
    log_table = log_table.append({'SEQ': Cont1,'LOG':'Ano: '+str(Ano)}, ignore_index=True)
    
    #Busca dados para o Ano em questão
    year_stocks = index_table[index_table['ANO'] == Ano]
    
    #Verifica se o Ano possui valores
    if len(year_stocks)==0:
        print("--Nao existe empresas para este ano")
    else:    
        
        #Ordenando por EV/EBIT
        year_stocks = year_stocks.sort_values('EV/EBIT',ascending=True)

        #Rankeando EV/EBIT
        index = 0
        count = 0
        for index, row in year_stocks.iterrows():
            count = count + 1
            year_stocks.loc[index,'ORD_EVEBIT'] = count

        #Ordenando por ROIC
        year_stocks = year_stocks.sort_values('ROIC',ascending=False) 

        #Rankeando ROIC
        index = 0
        count = 0
        for index, row in year_stocks.iterrows():
            count = count + 1
            year_stocks.loc[index,'ORD_ROIC'] = count

        #Somando os 2 rankeamentos
        index = 0
        count = 0
        for index, row in year_stocks.iterrows():
            count = count + 1
            year_stocks.loc[index,'RANKING'] = year_stocks.loc[index,'ORD_EVEBIT'] + year_stocks.loc[index,'ORD_ROIC']
        
        ###Begin - Retira Empresas repetidas, escolhendo a que possui menor cotacao - Begin
        year_stocks = year_stocks.sort_values('TICKER',ascending=True)        
        year_stocks['MANTER'] = 'Y'

        if RetiraRepCot == True:
            #Retira repetidas por cotacao    
            index0 = 0
            for index0, row0 in year_stocks.iterrows():
                TickerMenorCotacao = year_stocks.loc[index0,'TICKER']
                TickerNiv0 = ""
                TickerNiv1 = ""
                MenorCotacao = year_stocks.loc[index0,'COTACAO']
                index1 = 0
                for index1, row1 in year_stocks.iterrows():
                    TickerNiv0 = year_stocks.loc[index0,'TICKER']
                    TickerNiv1 = year_stocks.loc[index1,'TICKER']

                    if (TickerNiv0 != TickerNiv1) & (TickerNiv0[0:4] == TickerNiv1[0:4]):
                        if MenorCotacao > year_stocks.loc[index1,'COTACAO']:
                            TickerMenorCotacao = TickerNiv1
                            MenorCotacao = year_stocks.loc[index1,'COTACAO']
                        else:
                            year_stocks.loc[index1,'MANTER'] = 'N'
                        #end if
                    #end if
                #end for
            #end for
        else:
            #Retira repetidas por ranking     
            index0 = 0
            for index0, row0 in year_stocks.iterrows():
                TickerMenorRanking = year_stocks.loc[index0,'TICKER']
                TickerNiv0 = ""
                TickerNiv1 = ""
                MenorRanking = year_stocks.loc[index0,'RANKING']
                index1 = 0
                for index1, row1 in year_stocks.iterrows():
                    TickerNiv0 = year_stocks.loc[index0,'TICKER']
                    TickerNiv1 = year_stocks.loc[index1,'TICKER']

                    if (TickerNiv0 != TickerNiv1) & (TickerNiv0[0:4] == TickerNiv1[0:4]):
                        if MenorRanking > year_stocks.loc[index1,'RANKING']:
                            TickerMenorRanking = TickerNiv1
                            MenorRanking = year_stocks.loc[index1,'RANKING']
                        else:
                            year_stocks.loc[index1,'MANTER'] = 'N'
                        #end if
                    #end if
                #end for
            #end for
        #end if
        
        #Retira registros que nao devem ser mantidos
        year_stocks_prices = year_stocks
        
        #Volume maior do que Definido nos parâmetros
        year_stocks = year_stocks[year_stocks['VOL_MEDIO'] > Volume]
        
        year_stocks = year_stocks[year_stocks['MANTER'] == 'Y']
        ###End - Retira Empresas repetidas, escolhendo a que possui menor cotacao - End
        
        #Ordena tabela de rankeamento
        year_stocks = year_stocks.sort_values('RANKING',ascending=True)

        #Guarda quantidade de ações selecionadas do ano em questão
        range_stocks = year_stocks.head(QtdAcoes)[['TICKER','ANO','ROIC','EV','EV/EBIT','RANKING','NUM_ACOES','COTACAO']]
        range_stocks['NUM_ACOES_FIM_ANO'] = 0
        range_stocks['COTACAO_FIM_ANO'] = 0
        range_stocks['PRECO_INICIO_ANO'] = 0
        range_stocks['PRECO_FIM_ANO'] = 0
        range_stocks['STATUS'] = ''

        #Ordena tabela de Range
        range_stocks = range_stocks.sort_values('RANKING',ascending=True)

        #Guarda minhas ações do primeiro ano analisado
        if len(my_stocks) == 0:
            print(">> Ano Inicial")
            my_stocks = range_stocks
            my_stocks['PRECO_INICIO_ANO'] = my_stocks['EV']
            my_stocks['STATUS'] = 'COMPRADO'        
        else:                
            index = 0
            for index, row in my_stocks.iterrows():
                    
                #Considerar somente as acoes compradas do Ano Anterior da minha carteira 
                if (my_stocks.loc[index,'STATUS'] == 'COMPRADO')&(my_stocks.loc[index,'ANO'] == Ano-1):
                    print(">> Tratando Acoes do ano anterior: "+my_stocks.loc[index,'TICKER']+". Ano: "+str(my_stocks.loc[index,'ANO'])+". Status: "+my_stocks.loc[index,'STATUS'])
                    #Loga Programa
                    Cont1 += 1
                    log_table = log_table.append({'SEQ': Cont1,'LOG':'Ticker: '+my_stocks.loc[index,'TICKER']+', Ano: '+str(my_stocks.loc[index,'ANO'])+', Preço Final Ano: '}, ignore_index=True)

                    #Adiciona Preço do Final do ano para o ano anterior           
                    price_table = year_stocks_prices[year_stocks_prices['TICKER'] == my_stocks.loc[index,'TICKER']]
                    
                    #Verifica se existe acao no proximo ano na tabela para atribuir preco
                    if len(price_table)>0:
                        index2 = 0
                        for index2, row2 in price_table.iterrows():
                            print("$ Atualizando Preco: "+my_stocks.loc[index,'TICKER'])

                            my_stocks.loc[index,'COTACAO_FIM_ANO'] = price_table.loc[index2, 'COTACAO']
                            my_stocks.loc[index,'NUM_ACOES_FIM_ANO'] = price_table.loc[index2, 'NUM_ACOES']
                            my_stocks.loc[index,'PRECO_FIM_ANO'] = price_table.loc[index2, 'EV'] 
                        #end for
                    else:
                         print("$ Não existe preco no ao seguinte: "+my_stocks.loc[index,'TICKER'])
                            
                    #Verifica se minhas acoes aparecem no Ano Atual 
                    remain_stocks = range_stocks[range_stocks['TICKER'] == my_stocks.loc[index,'TICKER']]

                    #Atualiza Status para vendido caso nao tenha que permanecer em carteira      
                    if len(remain_stocks) == 0:
                        my_stocks.loc[index,'STATUS'] = 'VENDIDO'
                        print("xxxxxxxx Acao vendida: "+my_stocks.loc[index,'TICKER'])                      
                    else:
                        #Atualiza preco inicial e status no ano seguinte
                        remain_stocks['PRECO_INICIO_ANO'] = remain_stocks['EV']
                        remain_stocks['STATUS'] = 'COMPRADO'

                        print(">>>>>>>> Acao remanescente: "+my_stocks.loc[index,'TICKER'])
                        my_stocks = my_stocks.append(remain_stocks)

                        #Apaga do range as novas acoes que ja comprei
                        range_stocks  = range_stocks[range_stocks.TICKER != my_stocks.loc[index,'TICKER']]                  
                    #end if
                #end if
            #end for

            print("")
            print("--- Verifica quantas acoes faltam comprar")

            #Ordena tabela Range pelo Ranking
            range_stocks = range_stocks.sort_values('RANKING',ascending=True)

            #Verifica quantas acoes faltam comprar       
            bought_stocks_year = my_stocks[(my_stocks['STATUS'] == 'COMPRADO')&(my_stocks['ANO'] == Ano)]         
            TargetStockQty = QtdAcoes - len(bought_stocks_year)

            if TargetStockQty>0:
                print(">>Quantidade Faltante: "+str(TargetStockQty))
                new_stocks = range_stocks.head(TargetStockQty)

                #Atualiza preco inicial e status
                new_stocks['PRECO_INICIO_ANO'] = new_stocks['EV']
                new_stocks['STATUS'] = 'COMPRADO'

                #Adiciona Novas Ações
                my_stocks = my_stocks.append(new_stocks)
            #end if
        #end if
    #end while
    
#Atualiza Precos de acoes que nao existem no ano seguinte para o mesmo do ano anterior e monta resultado por empresa         
index3 = 0
print("--- TICKER Atualizando Preco zerados")
for index3, row3 in my_stocks.iterrows():

    if my_stocks.loc[index3,'COTACAO_FIM_ANO'] ==0:
        my_stocks.loc[index3,'COTACAO_FIM_ANO'] = my_stocks.loc[index3,'COTACAO']

    if my_stocks.loc[index3,'NUM_ACOES_FIM_ANO'] ==0:                            
        my_stocks.loc[index3,'NUM_ACOES_FIM_ANO'] = my_stocks.loc[index3,'NUM_ACOES']

    if my_stocks.loc[index3,'PRECO_FIM_ANO'] ==0: 
        my_stocks.loc[index3,'PRECO_FIM_ANO'] = my_stocks.loc[index3, 'EV']
        
    if GeraCotacaoRelatorio == True:
        #Resultado Utilizando a Cotacao - Analitico
        my_stocks['VALOR_RESULT_ANUAL_COT'] = my_stocks['COTACAO_FIM_ANO'] - my_stocks['COTACAO']
        my_stocks['PERC_RESULT_ANUAL_COT'] = (my_stocks['VALOR_RESULT_ANUAL_COT']*100)/my_stocks['COTACAO']
    
    if GeraPrecoRelatorio == True:
        #Resultado Utilizando o Enterprize Value - Analitico
        my_stocks['VALOR_RESULT_ANUAL_EV'] = my_stocks['PRECO_FIM_ANO'] - my_stocks['PRECO_INICIO_ANO']
        my_stocks['PERC_RESULT_ANUAL_EV'] = (my_stocks['VALOR_RESULT_ANUAL_EV']*100)/my_stocks['PRECO_INICIO_ANO']
#end for

print("--- Relatórios Resultado")
#Resultado Sintetico
if GeraCotacaoRelatorio == True:
    sintetic_result = my_stocks.groupby(['ANO'])['COTACAO', 'COTACAO_FIM_ANO'].sum()    
    #Resultado Utilizando a Cotacao - Sintetico
    sintetic_result['VALOR_RESULT_ANUAL_COT'] = sintetic_result['COTACAO_FIM_ANO'] - sintetic_result['COTACAO']
    sintetic_result['PERC_RESULT_ANUAL_COT'] = (sintetic_result['VALOR_RESULT_ANUAL_COT']*100)/ sintetic_result['COTACAO']

if GeraPrecoRelatorio == True & GeraCotacaoRelatorio == True:    
    sintetic_result = my_stocks.groupby(['ANO'])['COTACAO','COTACAO_FIM_ANO','PRECO_INICIO_ANO', 'PRECO_FIM_ANO'].sum()
    #Resultado Utilizando o Enterprize Value - Sintetico
    sintetic_result['VALOR_RESULT_ANUAL_EV'] = sintetic_result['PRECO_FIM_ANO'] - sintetic_result['PRECO_INICIO_ANO']
    sintetic_result['PERC_RESULT_ANUAL_EV'] = (sintetic_result['VALOR_RESULT_ANUAL_EV']*100)/ sintetic_result['PRECO_INICIO_ANO']
    
    #Resultado Utilizando a Cotacao - Sintetico
    sintetic_result['VALOR_RESULT_ANUAL_COT'] = sintetic_result['COTACAO_FIM_ANO'] - sintetic_result['COTACAO']
    sintetic_result['PERC_RESULT_ANUAL_COT'] = (sintetic_result['VALOR_RESULT_ANUAL_COT']*100)/ sintetic_result['COTACAO']

if GeraCotacaoRelatorio == True:
    analytic_result = my_stocks[['TICKER','ANO','COTACAO','COTACAO_FIM_ANO','VALOR_RESULT_ANUAL_COT','PERC_RESULT_ANUAL_COT']]

if GeraPrecoRelatorio == True:  
    analytic_result = my_stocks[['TICKER','ANO','PRECO_INICIO_ANO', 'PRECO_FIM_ANO','VALOR_RESULT_ANUAL_EV','PERC_RESULT_ANUAL_EV','COTACAO','COTACAO_FIM_ANO','VALOR_RESULT_ANUAL_COT','PERC_RESULT_ANUAL_COT']]

#Exporta o Excel Resultado Anual
sintetic_result.to_excel("resultado_sintetico_"+str(QtdAcoes)+"_acoes_"+str(Volume)+"_volume.xlsx")

#Resultado Analitico
analytic_result.to_excel("resultado_analitico_"+str(QtdAcoes)+"_acoes_"+str(Volume)+"_volume.xlsx")

print("FINAL")

 
>>>>>>>>>>>>>>>Ano: 2009
--Nao existe empresas para este ano
 
>>>>>>>>>>>>>>>Ano: 2010
>> Ano Inicial
 
>>>>>>>>>>>>>>>Ano: 2011
>> Tratando Acoes do ano anterior: VALE3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: VALE3
>>>>>>>> Acao remanescente: VALE3


C:\Users\filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


>> Tratando Acoes do ano anterior: MTSA4. Ano: 2010. Status: COMPRADO
$ Não existe preco no ao seguinte: MTSA4
xxxxxxxx Acao vendida: MTSA4
>> Tratando Acoes do ano anterior: EQTL3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: EQTL3
>>>>>>>> Acao remanescente: EQTL3
>> Tratando Acoes do ano anterior: CIEL3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: CIEL3
>>>>>>>> Acao remanescente: CIEL3
>> Tratando Acoes do ano anterior: RAPT3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: RAPT3
>>>>>>>> Acao remanescente: RAPT3
>> Tratando Acoes do ano anterior: CGRA3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: CGRA3
xxxxxxxx Acao vendida: CGRA3
>> Tratando Acoes do ano anterior: CSMG3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: CSMG3
xxxxxxxx Acao vendida: CSMG3
>> Tratando Acoes do ano anterior: LIGT3. Ano: 2010. Status: COMPRADO
$ Atualizando Preco: LIGT3
xxxxxxxx Acao vendida: LIGT3
>> Tratando Acoes do ano anterior: BRAP4. Ano: 2010. Status: COMPRADO
$ Atualizando Preco

>> Tratando Acoes do ano anterior: BEEF3. Ano: 2012. Status: COMPRADO
$ Atualizando Preco: BEEF3
>>>>>>>> Acao remanescente: BEEF3
>> Tratando Acoes do ano anterior: BRML3. Ano: 2012. Status: COMPRADO
$ Atualizando Preco: BRML3
>>>>>>>> Acao remanescente: BRML3
>> Tratando Acoes do ano anterior: ECOR3. Ano: 2012. Status: COMPRADO
$ Atualizando Preco: ECOR3
>>>>>>>> Acao remanescente: ECOR3
>> Tratando Acoes do ano anterior: NATU3. Ano: 2012. Status: COMPRADO
$ Atualizando Preco: NATU3
>>>>>>>> Acao remanescente: NATU3
>> Tratando Acoes do ano anterior: CEDO4. Ano: 2012. Status: COMPRADO
$ Não existe preco no ao seguinte: CEDO4
xxxxxxxx Acao vendida: CEDO4
>> Tratando Acoes do ano anterior: SHUL4. Ano: 2012. Status: COMPRADO
$ Atualizando Preco: SHUL4
xxxxxxxx Acao vendida: SHUL4
>> Tratando Acoes do ano anterior: HAGA4. Ano: 2012. Status: COMPRADO
$ Atualizando Preco: HAGA4
xxxxxxxx Acao vendida: HAGA4
>> Tratando Acoes do ano anterior: ETER3. Ano: 2012. Status: COMPRADO
$ Atualizando 

$ Atualizando Preco: HGTX3
xxxxxxxx Acao vendida: HGTX3
>> Tratando Acoes do ano anterior: ITSA4. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: ITSA4
>>>>>>>> Acao remanescente: ITSA4
>> Tratando Acoes do ano anterior: CMIG4. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: CMIG4
>>>>>>>> Acao remanescente: CMIG4
>> Tratando Acoes do ano anterior: MGLU3. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: MGLU3
>>>>>>>> Acao remanescente: MGLU3
>> Tratando Acoes do ano anterior: RAPT3. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: RAPT3
xxxxxxxx Acao vendida: RAPT3
>> Tratando Acoes do ano anterior: RENT3. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: RENT3
>>>>>>>> Acao remanescente: RENT3
>> Tratando Acoes do ano anterior: ENBR3. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: ENBR3
>>>>>>>> Acao remanescente: ENBR3
>> Tratando Acoes do ano anterior: MYPK3. Ano: 2014. Status: COMPRADO
$ Atualizando Preco: MYPK3
xxxxxxxx Acao vendida: MYPK3
>> Tratando Acoes do ano anterior: BR

>>>>>>>> Acao remanescente: ITSA3
>> Tratando Acoes do ano anterior: CCXC3. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: CCXC3
xxxxxxxx Acao vendida: CCXC3
>> Tratando Acoes do ano anterior: VVAR3. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: VVAR3
>>>>>>>> Acao remanescente: VVAR3
>> Tratando Acoes do ano anterior: VALE3. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: VALE3
>>>>>>>> Acao remanescente: VALE3
>> Tratando Acoes do ano anterior: GEPA4. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: GEPA4
xxxxxxxx Acao vendida: GEPA4
>> Tratando Acoes do ano anterior: LOGN3. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: LOGN3
xxxxxxxx Acao vendida: LOGN3
>> Tratando Acoes do ano anterior: POSI3. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: POSI3
xxxxxxxx Acao vendida: POSI3
>> Tratando Acoes do ano anterior: ELET3. Ano: 2016. Status: COMPRADO
$ Atualizando Preco: ELET3
xxxxxxxx Acao vendida: ELET3
>> Tratando Acoes do ano anterior: CCRO3. Ano: 2016. Status: COMPRADO
$ At

In [6]:
log_table.to_excel("log_formula_magica_"+now.strftime("%Y-%m-%d")+".xlsx")